In [1]:
import pandas as pd
import numpy as np
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle, NCBITaxa, random_color, TextFace, CircleFace, RectFace
import random
import glob

C:\Users\cmgro\anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\cmgro\anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [3]:
####reference.csv is your genome group csv from patric 
df =pd.read_csv("C:\\Users\\cmgro\\Downloads\\genome_group_gi_strains.csv", dtype={'Genome ID': str})

In [30]:
def parse_blast_results(file_path, cov=50, identity=30):
    temp_df = pd.read_csv(file_path, dtype={'Genome ID': str})

    temp_df = temp_df[(temp_df["Identity"] > identity) &(temp_df["Query cover"] > cov)]
    return set(temp_df["Genome ID"])
#####Create one for each of your genes 
gadA_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadA.csv", identity=30, cov=50) 
gadB_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadB.csv", identity=30, cov=50)
gadC_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadC.csv", identity=30, cov=50)
gadE_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadE.csv", identity=20, cov=30)
gadX_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadX.csv", identity=30, cov=50)
gadW_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_gadW.csv", identity=30, cov=50)
adiA_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_adiA.csv", identity=30, cov=50)
adiC_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_adiC.csv", identity=30, cov=50)
adiY_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_adiY.csv", identity=30, cov=50)
CsrA_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_CsrA.csv", identity=40)
CsrD_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_CsrD.csv", identity=20, cov=50)
sdiA_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\merged_sdiA.csv", identity=20, cov=30)
#These parameters look funky and they are and are subject to change
fliw_genomes = parse_blast_results("C:\\Users\\cmgro\\Downloads\\mark_ex3.csv", identity=30, cov=30)
print(fliw_genomes)

{'665952.3', '585538.3'}


In [31]:
#Create an object and assign it to the variable 
ncbi = NCBITaxa()


#Dataframes are cool becuase becuase you can take a spreadsheet and slice out a whole column - trust me this is super useful 
ids = set(df["NCBI Taxon ID"]) #use set to remove duplicates

#Strains
patric_id = list(df["Genome ID"])
strain_name = list(df["Genome Name"])
#Extract the tree
tree = ncbi.get_topology(ids, intermediate_nodes=True)
tree.get_ascii(attributes=["sci_name"])


lineages = {}
for i in tree.traverse():
    if len(i.named_lineage) > 5:
        if i.named_lineage[4] in lineages.keys():
            lineages[i.named_lineage[4]].append(i.name)
        else:
            lineages[i.named_lineage[4]] = [i.name]


node_groups = {}
lineage_colors = {"Firmicutes":"Aquamarine", "Bacteroidetes/Chlorobi group":"LightCyan", "delta/epsilon subdivisions":"LightPink", 
"Gammaproteobacteria":"Gold",  "Fusobacteriia":"Violet", "Actinobacteria":"MistyRose", "Betaproteobacteria":"Gainsboro", "Alphaproteobacteria": "red",
                 "Spirochaetia": "orange", "Verrucomicrobia": "yellow"}

for j in lineages.keys():
    node_groups[j] = tree.get_common_ancestor(lineages[j])
    temp_nest = NodeStyle()
    temp_nest["bgcolor"] = lineage_colors[j]
    node_groups[j].set_style(temp_nest)

for position, i in enumerate(patric_id):

    if len(tree.search_nodes(name=i.split(".")[0])) > 0:
            node = tree.search_nodes(name=i.split(".")[0])[0]
    node.add_child(name=i)
    for j in tree.search_nodes(name=i):
        j.add_features(label="sci_name")
        j.sci_name = strain_name[position]



def layout(node):
    if node.is_leaf():
        N = AttrFace("sci_name", fsize=30)
        faces.add_face_to_node(N, node, 0, position="branch-right")
        #####add an if statement for each of your genes, each column needs to be one higher than the one before it and make sure to use inuqie colors
        #####You can make a seperate tree for each "block" or operon you are looking at, just amke sure to include csrA and csrD
        if node.name in gadA_genomes:
            circle = RectFace(60, 60, "black","red")
            faces.add_face_to_node(circle, position="aligned", column=1, node=node)
        if node.name in gadB_genomes:
            circle = RectFace(60, 60, "black","black")
            faces.add_face_to_node(circle, position="aligned", column=2, node=node)
        if node.name in gadC_genomes:
            circle = RectFace(60, 60, "black","cyan")
            faces.add_face_to_node(circle, position="aligned", column=3, node=node)
        if node.name in gadE_genomes:
            circle = RectFace(60, 60, "black","yellow")
            faces.add_face_to_node(circle, position="aligned", column=4, node=node)
        if node.name in gadW_genomes:
            circle = RectFace(60, 60, "lightgreen","lightgreen")
            faces.add_face_to_node(circle, position="aligned", column=5, node=node)
        if node.name in gadX_genomes:
            circle = RectFace(60, 60, "purple","purple")
            faces.add_face_to_node(circle, position="aligned", column=6, node=node)
        if node.name in adiA_genomes:
            circle = RectFace(60, 60, "gray","gray")
            faces.add_face_to_node(circle, position="aligned", column=7, node=node)
        if node.name in adiC_genomes:
            circle = RectFace(60, 60, "black","orange")
            faces.add_face_to_node(circle, position="aligned", column=8, node=node)
        if node.name in adiY_genomes:
            circle = RectFace(60, 60, "black","lightblue")
            faces.add_face_to_node(circle, position="aligned", column=9, node=node)
        if node.name in CsrA_genomes:
            circle = RectFace(60, 60, "darkblue","darkblue")
            faces.add_face_to_node(circle, position="aligned", column=10, node=node)
        if node.name in CsrD_genomes:
            circle = RectFace(60, 60, "darkgreen","darkgreen")
            faces.add_face_to_node(circle, position="aligned", column=11, node=node)
        if node.name in sdiA_genomes:
            circle = RectFace(60, 60, "black","darkred")
            faces.add_face_to_node(circle, position="aligned", column=12, node=node)
        # Adding Fliw since it implies if CsrA is present that it plays a different regulatory role than if csrD is present
        if node.name in fliw_genomes:
            circle = RectFace(60, 60, "black","pink")
            faces.add_face_to_node(circle, position="aligned", column=13, node=node)
                # Adding Fliw since it implies if CsrA is present that it plays a different regulatory role than if csrD is present
       
ts = TreeStyle()
ts.branch_vertical_margin = 10
ts.allow_face_overlap = False
ts.show_scale = False
ts.show_leaf_name = False
ts.layout_fn = layout
ts.mode = "c"
ts.root_opening_factor = .25
ts.optimal_scale_level = "full"
ts.force_topology = True
for i, j in enumerate(lineage_colors.keys()):
    ts.legend.add_face(RectFace(500, 30, lineage_colors[j], lineage_colors[j]), column=i)
    ts.legend.add_face(TextFace(j, fsize=30), column=i)
tree.render("firsttree.pdf", tree_style=ts)
print("Done")

Done
